# Some solutions for the prep workshop

## SVD

In [ ]:
def get_pseudo_inv(X):
    # Perform SVD on the matrix X
    U, S, V_t = svd(X)
    print(f'Singular values: {S}\nU: {U}\nV_t: {V_t}')

    # Calculate Pseudo Inverse
    S_inv = np.zeros(X.shape)
    for i, s_val in enumerate(S):
        S_inv[i, i] = 1 / s_val
    
    M = (V_t.T @ S_inv.T) @ U.T

    return M 

# Let's test this function
X = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
get_pseudo_inv(X)

In [ ]:
sample = iio.imread(uri="exercise.png")

fig, ax = plt.subplots()

grey_sample = rgb2gray(sample)

U, S, V_T = svd(grey_sample, full_matrices=False)
S = np.diag(S)
fig, ax = plt.subplots(7, 2, figsize=(8, 20))


curr_fig = 0
for r in [2,4,8,16,32,64,128]:
    approx = U[:, :r] @ S[0:r, :r] @ V_T[:r, :]
    ax[curr_fig][0].imshow(approx, cmap='gray')
    ax[curr_fig][0].set_title("k = "+str(r))
    ax[curr_fig, 0].axis('off')
    ax[curr_fig][1].set_title("Original Image")
    ax[curr_fig][1].imshow(grey_sample, cmap='gray')
    ax[curr_fig, 1].axis('off')
    curr_fig += 1
plt.show()

## Lyapunov

In [ ]:
# Import Libraries
import numpy as np
import matplotlib.pyplot as plt

def logistic_map(x, r):
    """Logistic map function"""
    return r * x * (1 - x)

def lyapunov_exponent(r, x0, n_iterations, n_discard=100):
    """Calculate Lyapunov exponent using the direct method"""
    x = x0
    le = 0.0
    delta = 1e-10  # Small perturbation

    val = []
    # Discard initial transients
    for _ in range(n_discard):
      x = logistic_map(x, r)

    # Main loop for Lyapunov exponent calculation
    for i in range(n_iterations):
      x_perturbed = x + delta
      
      # Evolve both trajectories one step
      x_new = logistic_map(x, r)
      x_perturbed_new = logistic_map(x_perturbed, r)
      
      # Calculate new separation
      delta_new = abs(x_perturbed_new - x_new)
      
      # Update Lyapunov exponent
      le += np.log(abs(delta_new / delta))
      
      # Renormalize
      x = x_new
      delta = delta_new / abs(delta_new) * delta
      if i >= n_iterations - 300:
        val.append(x)
    return [le / n_iterations, val]

# Parameters
r_range = np.linspace(2.5, 4.0, 1000)
x0 = 0.5
n_iterations = 10000
n_discard = 100

# Calculate Lyapunov exponents for different r values
data = [lyapunov_exponent(r, x0, n_iterations, n_discard) for r in r_range]
lyapunov_exponents = [d[0] for d in data]
x_vals = [d[1] for d in data]

# Plot the results
mosaic_layout = '''
AAAA
AAAA
AAAA
....
BBBB
BBBB
BBBB
'''

fig, ax = plt.subplot_mosaic(mosaic_layout, layout = 'constrained', sharex = True)
ax['A'].plot(r_range, lyapunov_exponents)
ax['B'].set_xlabel('r')
ax['A'].set_ylabel('$\lambda$', rotation = 0, labelpad = 10)
ax['B'].plot(r_range, x_vals, 'k,', alpha=0.25)

ax['B'].set_ylabel('$x_{n+1}$', rotation = 0, labelpad = 20)

ax['A'].grid(True)
ax['B'].grid(True)
plt.show()